In [1]:
import math
import warnings
from typing import TYPE_CHECKING, Optional

import random
import numpy as np

import gymnasium as gym
from gymnasium import error, spaces
from gymnasium.error import DependencyNotInstalled
from gymnasium.utils import EzPickle, colorize
from gymnasium.utils.step_api_compatibility import step_api_compatibility

import pygame

import Box2D
from Box2D.b2 import (
    circleShape,
    contactListener,
    edgeShape,
    fixtureDef,
    polygonShape,
    revoluteJointDef,
    staticBody, 
    dynamicBody
)

import pygame

import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
import optax
from collections import deque
import pickle

### **state Space**
$(x, y, p_x, p_y, angle, L, mass, intertia)$ : most simplified

### **observation**
(FOV array)

### **Action space**
(Propulsion (acc in facing angle), angular velocity)

### **Rewards**
*   increase/decrease the closer/durther the agent is to the goal.
*   increase/decrease the larger/smaller the magnitute of velocity $||v||$.
*   increase/decrease the larger/smaller the $\%$ of obstacle in FOV.

The episode receive an additional reward of -100 for crashing.

An episode is considered a solution if it reaches score.

### **Starting State**
The agent starts at the left ceter of the viewpoirt, with a initial $v_x$

### **Episode Termination**
If:
* The agent crashes
* The agent gets outside of the viewport

In [2]:
try:
    env.close()
except:
    pass

# Constants
FPS = 50
SCALE = 10.0  # Scaling for visualization
OBSTACLE_SCALE = 5.0  # Scaling for obstacles
UAV_RADIUS = 10.0 / SCALE  # Radius of the UAV
GOAL_RADIUS = UAV_RADIUS  # Radius of the Goal
VIEWPORT_W = 600
VIEWPORT_H = 400
WALL_THICKNESS = 1.0  # Thickness of the walls
NUM_OBSTACLES = 10  # Number of obstacles
OBS_MAX_RAD = min(VIEWPORT_W, VIEWPORT_H) / SCALE / OBSTACLE_SCALE # Maximum radius of obstacles
OBS_MIN_RAD = OBS_MAX_RAD / 3  # Minimum radius of obstacles
MIN_CLEARANCE = UAV_RADIUS * 3  # Minimum clearance between obstacles

#Note: For the angle in box2d, 0 rad is at 3 o'clock, and positive angle is in the clockwise direction

# UAV specs
UAV_INI_ANGLE = np.deg2rad(0)
UAV_DENSITY = 1.0
UAV_FRICTION = 0.3
UAV_FOV = np.deg2rad(90)  # Field of View in degrees
UAV_NUM_RAYS = 10  # Number of rays in the FOV array
UAV_FOV_DISTANCE = 100 / SCALE  # Maximum sensing distance
UAV_ANG_POW = 10  # Maximum angular velocity (deg/s?)
UAV_THRUST_POW = 10  # Maximum thrust (Unit/s?)

# Penalty/Reward coeff
PEN_THRUST = -0.1
PEN_ANG = -0.01
PEN_OBSTACLE = -1
PEN_COLLISION = -100
REW_VEL = 1
REW_ANGLE = 0.3
REW_GOAL = 100
REW_DIST2GOAL = 10

class ContactDetector(Box2D.b2ContactListener):
    def __init__(self, env):
        super(ContactDetector, self).__init__()
        self.env = env

    def BeginContact(self, contact):
        # Check if one of the bodies is the UAV and the other is an obstacle or wall
        if contact.fixtureA.body == self.env.uav or contact.fixtureB.body == self.env.uav:
            self.env.game_over = True

class SimpleUAVEnv(gym.Env):
    metadata = {'render_modes': ['human'], 'render_fps': FPS}

    def __init__(self):
        # Define the Box2D world
        self.world = Box2D.b2World(gravity=(0, 0))
        self.uav = None  # UAV object
        self.goal = None  # Goal position
        self.dist2goal = None  # Distance to goal
        self.walls = []  # Walls
        self.obstacles = []  # Store properties of obstacles
        self.obstacles_properties = []


        # For rendering
        self.screen = None
        self.isopen = True

    def _create_walls(self):
        # Create walls around the viewport
        wall_shapes = [
            edgeShape(vertices=[(0, 0), (VIEWPORT_W/SCALE, 0)]),  # Bottom
            edgeShape(vertices=[(0, 0), (0, VIEWPORT_H/SCALE)]),  # Left
            edgeShape(vertices=[(0, VIEWPORT_H/SCALE), (VIEWPORT_W/SCALE, VIEWPORT_H/SCALE)]),  # Top
            edgeShape(vertices=[(VIEWPORT_W/SCALE, 0), (VIEWPORT_W/SCALE, VIEWPORT_H/SCALE)])  # Right
        ]
        for shape in wall_shapes:
            wall_body = self.world.CreateStaticBody(
                position=(0, 0),
                shapes=shape
            )
            self.walls.append(wall_body)

    def _is_position_valid(self, new_properties):
        # Check against existing obstacles
        for prop in self.obstacles_properties:
            distance = math.sqrt((new_properties['centroid_x'] - prop['centroid_x'])**2 + 
                                (new_properties['centroid_y'] - prop['centroid_y'])**2)
            if distance < (new_properties['max_span'] + prop['max_span'] + MIN_CLEARANCE):
                return False

        # Check distance to the goal
        goal_distance = math.sqrt((new_properties['centroid_x'] - self.goal[0])**2 +
                                (new_properties['centroid_y'] - self.goal[1])**2)
        if goal_distance < (new_properties['max_span'] + 1.5*UAV_RADIUS):
            return False

        # Check distance to the uav starting position
        uav_distance = math.sqrt((new_properties['centroid_x'] - self.uav_start_pos[0])**2 +
                                (new_properties['centroid_y'] - self.uav_start_pos[1])**2)
        if uav_distance < (new_properties['max_span'] + 1.5*UAV_RADIUS):
            return False

        return True
    
    def _generate_triangle_properties(self):
        # Randomly generate centroid within bounds
        centroid_x = random.uniform(UAV_RADIUS, VIEWPORT_W / SCALE - UAV_RADIUS)
        centroid_y = random.uniform(UAV_RADIUS, VIEWPORT_H / SCALE - UAV_RADIUS)
        
        # Random length from centroid to vertices
        length = random.uniform(OBS_MIN_RAD, OBS_MAX_RAD)

        # Angle offsets for equilateral triangle vertices
        angle_offset = math.pi * 2 / 3

        # Calculate vertices based on centroid and length
        vertices = []
        for i in range(3):
            angle = angle_offset * i
            vertex_x = centroid_x + length * math.cos(angle)
            vertex_y = centroid_y + length * math.sin(angle)
            vertices.append((vertex_x, vertex_y))

        # Random rotation angle
        rotation_angle = random.uniform(0, math.pi)

        return {'type': 'triangle', 'vertices': vertices, 'centroid_x': centroid_x, 'centroid_y': centroid_y, 'angle': rotation_angle, 'max_span': length}

    def _generate_rectangle_properties(self):
        width = random.uniform(OBS_MIN_RAD, OBS_MAX_RAD)
        height = random.uniform(OBS_MIN_RAD, OBS_MAX_RAD)
        angle = random.uniform(0, math.pi)  # Random angle in radians

        centroid_x = random.uniform(width / 2, VIEWPORT_W / SCALE - width / 2)
        centroid_y = random.uniform(height / 2, VIEWPORT_H / SCALE - height / 2)

        return {'type': 'rectangle', 'centroid_x': centroid_x, 'centroid_y': centroid_y, 'width': width, 'height': height, 'angle': angle, 'max_span': max(width, height)}

    def _generate_circle_properties(self):
        radius = random.uniform(OBS_MIN_RAD, OBS_MAX_RAD)
        centroid_x = random.uniform(radius, VIEWPORT_W / SCALE - radius)
        centroid_y = random.uniform(radius, VIEWPORT_H / SCALE - radius)

        return {'type': 'circle', 'centroid_x': centroid_x, 'centroid_y': centroid_y, 'max_span': radius}

    def _create_obstacle_from_properties(self, properties):
        if properties['type'] == 'circle':
            body = self.world.CreateStaticBody(position=(properties['centroid_x'], properties['centroid_y']))
            circle = body.CreateCircleFixture(radius=properties['max_span'], density=1, friction=0.3)
            self.obstacles.append(circle)

        elif properties['type'] == 'rectangle':
            body = self.world.CreateStaticBody(position=(properties['centroid_x'], properties['centroid_y']))
            rectangle = body.CreatePolygonFixture(box=(properties['width'] / 2, properties['height'] / 2), density=1, friction=0.3)
            body.angle = properties['angle']
            self.obstacles.append(rectangle)

        elif properties['type'] == 'triangle':
            vertices = [(v[0] - properties['centroid_x'], v[1] - properties['centroid_y']) for v in properties['vertices']]
            body = self.world.CreateStaticBody(position=(properties['centroid_x'], properties['centroid_y']))
            triangle = body.CreatePolygonFixture(vertices=vertices, density=1, friction=0.3)
            body.angle = properties['angle']
            self.obstacles.append(triangle)

    def _create_obstacles(self, num_obstacles=5):
        num_obstacles = num_obstacles
        obstacle_types = ['triangle', 'rectangle', 'circle']
        max_iter = 1000
        for _ in range(num_obstacles):
            obstacle_type = random.choice(obstacle_types)
            for iter in range(max_iter):
                if obstacle_type == 'triangle':
                    properties = self._generate_triangle_properties()
                elif obstacle_type == 'rectangle':
                    properties = self._generate_rectangle_properties()
                elif obstacle_type == 'circle':
                    properties = self._generate_circle_properties()

                # Check if the new obstacle overlaps with existing ones
                if self._is_position_valid(properties):
                    self.obstacles_properties.append(properties)
                    # Create the actual obstacle based on properties
                    #print(properties['type']+': pass checking')
                    self._create_obstacle_from_properties(properties)
                    break
                if iter == max_iter - 1:
                    print('Failed to create: ', obstacle_type)

    def _create_uav(self):
        # Create the UAV at a position away from the left wall
        uav_start_pos = (UAV_RADIUS + 2 * WALL_THICKNESS, VIEWPORT_H / SCALE / 2)
        self.uav_start_pos = uav_start_pos
        self.uav = self.world.CreateDynamicBody(position=uav_start_pos, angle=UAV_INI_ANGLE, linearVelocity=(0,0), angularVelocity=0.0)
        self.uav.CreateCircleFixture(radius=UAV_RADIUS, density=UAV_DENSITY, friction=UAV_FRICTION)

    def _create_goal(self):
        # Create a random goal position away from the right wall
        goal_pos_x = VIEWPORT_W / SCALE - GOAL_RADIUS - 2 * WALL_THICKNESS
        goal_pos_y = random.uniform(WALL_THICKNESS+UAV_RADIUS, VIEWPORT_H / SCALE - UAV_RADIUS)
        self.goal = (goal_pos_x, goal_pos_y)
        self.ini_to_goal_dist = math.sqrt((self.uav_start_pos[0] - self.goal[0])**2 + (self.uav_start_pos[1] - self.goal[1])**2)

    def _calculate_reward(self, obs, action):
        # Distance to goal
        dist2goal = math.sqrt((self.uav.position.x - self.goal[0])**2 + (self.uav.position.y - self.goal[1])**2)
        self.dist2goal = dist2goal
        distance_reward = (1 - dist2goal / self.ini_to_goal_dist) * REW_DIST2GOAL  # Normalize 

        # Velocity reward
        velocity_reward = self.uav.linearVelocity.length * REW_VEL

        # Angle reward
        angle_reward = (1 - abs(self.uav.angle - np.arccos((self.goal[1]-self.uav.position.x)/dist2goal))/(2*math.pi)) * REW_ANGLE

        # FOV obstacle percentage
        fov_reward = (np.sum(obs / UAV_FOV_DISTANCE) / UAV_NUM_RAYS) * PEN_OBSTACLE

        # Active penalty
        act_reward = PEN_THRUST * abs(action[0]) + PEN_ANG * abs(action[1])

        return distance_reward, velocity_reward, angle_reward, fov_reward, act_reward, distance_reward

    def step(self, action):
        assert self.uav is not None, "You forgot to call reset()"
        thrust, angular_momentum = action
        # Calculate the thrust direction based on the UAV's angle, note the thrust is implemented via impulse (reverse)
        thrust_direction = Box2D.b2Vec2(math.cos(self.uav.angle), math.sin(self.uav.angle))
        thrust = np.clip(thrust, 0., 1.)
        angular_momentum = np.clip(angular_momentum, -1., 1.) * UAV_ANG_POW
        thrust_force = thrust_direction * thrust * UAV_THRUST_POW

        # Apply the thrust & angular momentum
        self.uav.ApplyLinearImpulse(thrust_force, self.uav.worldCenter, True)
        self.uav.ApplyTorque((angular_momentum), True)

        # Update the environment state
        self.world.Step(1.0 / FPS, 6 * 30, 2 * 30)  # Advance the Box2D world, Step (float timeStep, int32 velocityIterations, int32 positionIterations)
        # Get new observation
        new_obs = self._get_obs()
        # Calculate reward
        distance_reward, velocity_reward, angle_reward, fov_reward, act_reward, distance_reward = self._calculate_reward(new_obs, action)
        reward = distance_reward + velocity_reward + angle_reward + fov_reward + act_reward
        # Check if episode is done
        done = False
        if self.game_over:
            done = True
            reward += PEN_COLLISION
        if self.dist2goal <= GOAL_RADIUS:
            done = True
            reward += REW_GOAL
        # Additional info (optional)
        raw_reward = np.array((distance_reward/REW_DIST2GOAL,  velocity_reward/REW_VEL, angle_reward/REW_ANGLE, fov_reward/PEN_OBSTACLE, act_reward))
        pos = self.uav.position
        vel = self.uav.linearVelocity

        state = [
            (pos.x - VIEWPORT_W / SCALE / 2) / (VIEWPORT_W / SCALE / 2),
            (pos.y - VIEWPORT_H / SCALE / 2) / (VIEWPORT_H / SCALE / 2),
            vel.x * (VIEWPORT_W / SCALE / 2) / FPS,
            vel.y * (VIEWPORT_H / SCALE / 2) / FPS,
            self.uav.angle,
            20.0 * self.uav.angularVelocity / FPS,
            *new_obs/UAV_FOV_DISTANCE
        ]

        return state, reward, done, raw_reward


    def reset(self):
        # Reset the environment to initial state
        self._create_walls()
        self._create_uav()
        self._create_goal()
        self._create_obstacles(NUM_OBSTACLES)  
        self.game_over = False  
        self.world.contactListener = ContactDetector(self)
        # Destroy the UAV if it exists
        if self.uav:
            self.world.DestroyBody(self.uav)

        # Destroy existing obstacles
        for obstacle in self.obstacles:
            if obstacle.body:  # Ensure the body exists before destroying
                self.world.DestroyBody(obstacle.body)

        # Clear the lists of obstacles and their properties
        self.obstacles.clear()
        self.obstacles_properties.clear()

        # Recreate the UAV and the goal
        self._create_uav()
        self._create_goal()

        # Recreate obstacles
        self._create_obstacles(NUM_OBSTACLES)
        self.game_over = False

        return self.step(np.array([0., 0.]))[0]

    def _raycast_distance(self, start_pos, angle, max_distance):
        # Calculate the end point of the ray
        end_pos = (start_pos.x + max_distance * math.cos(angle), 
                start_pos.y + max_distance * math.sin(angle))

        # Define a callback class to record ray hits
        class RayCastCallback(Box2D.b2RayCastCallback):
            def __init__(self):
                super(RayCastCallback, self).__init__()
                self.fixture = None
                self.point = None
                self.normal = None

            def ReportFixture(self, fixture, point, normal, fraction):
                self.fixture = fixture
                self.point = Box2D.b2Vec2(point)
                self.normal = Box2D.b2Vec2(normal)
                return fraction  # Returning the fraction leaves the ray cast going to max_distance

        # Create a raycast callback instance
        callback = RayCastCallback()

        # Cast the ray
        self.world.RayCast(callback, start_pos, end_pos)

        # If a hit was recorded, calculate the distance, else return max_distance
        if callback.fixture:
            hit_position = callback.point
            distance = math.sqrt((hit_position.x - start_pos.x)**2 + (hit_position.y - start_pos.y)**2)
            return distance
        else:
            return max_distance

    def _get_obs(self):
        fov_array = np.zeros(UAV_NUM_RAYS)

        start_angle = self.uav.angle - UAV_FOV / 2
        angle_increment = UAV_FOV / UAV_NUM_RAYS

        for i in range(UAV_NUM_RAYS):
            ray_angle = start_angle + i * angle_increment
            distance = self._raycast_distance(self.uav.position, ray_angle, UAV_FOV_DISTANCE)
            fov_array[i] = distance

        return fov_array

    def render(self, mode='human'):
        if self.screen is None:
            pygame.init()
            self.screen = pygame.display.set_mode((VIEWPORT_W, VIEWPORT_H))

        # Clear screen
        self.screen.fill((255, 255, 255))

        # Draw the UAV
        for body in [self.uav]:  # Currently, we only have the UAV
            for fixture in body.fixtures:
                shape = fixture.shape
                position = body.transform * shape.pos * SCALE
                position = (position[0], VIEWPORT_H - position[1])  # Flip Y
                pygame.draw.circle(self.screen, (255, 0, 0), [int(x) for x in position], int(shape.radius * SCALE))

        # Draw the goal
        goal_position = (self.goal[0] * SCALE, VIEWPORT_H - self.goal[1] * SCALE)
        pygame.draw.circle(self.screen, (0, 255, 0), [int(x) for x in goal_position], int(GOAL_RADIUS * SCALE))

        # Draw the obstacles
        for fixture in self.obstacles:
            shape = fixture.shape
            body = fixture.body

            if isinstance(shape, Box2D.b2CircleShape):
                # For circle shapes
                position = (body.position.x * SCALE, VIEWPORT_H - body.position.y * SCALE)
                pygame.draw.circle(self.screen, (0, 0, 255), [int(x) for x in position], int(shape.radius * SCALE))
            
            elif isinstance(shape, Box2D.b2PolygonShape):
                # For polygon shapes (rectangles, triangles)
                vertices = [(body.transform * v) * SCALE for v in shape.vertices]
                vertices = [(v[0], VIEWPORT_H - v[1]) for v in vertices]
                pygame.draw.polygon(self.screen, (0, 0, 255), vertices)

        wall_color = (0, 0, 0)  # Black color for walls
        for wall in self.walls:
            for fixture in wall.fixtures:
                shape = fixture.shape
                # Since these are edge shapes, they have exactly two vertices
                vertex1, vertex2 = shape.vertices
                vertex1 = (wall.transform * vertex1) * SCALE
                vertex2 = (wall.transform * vertex2) * SCALE
                vertex1 = (vertex1[0], VIEWPORT_H - vertex1[1])  # Flip Y
                vertex2 = (vertex2[0], VIEWPORT_H - vertex2[1])  # Flip Y
                pygame.draw.line(self.screen, wall_color, vertex1, vertex2, int(WALL_THICKNESS * SCALE))

        start_angle = self.uav.angle -UAV_FOV / 2
        angle_increment = UAV_FOV / UAV_NUM_RAYS
        for i in range(UAV_NUM_RAYS):
            ray_angle = start_angle + i * angle_increment
            distance = self._raycast_distance(self.uav.position, ray_angle, UAV_FOV_DISTANCE)
            end_x = self.uav.position.x + distance * math.cos(ray_angle)
            end_y = self.uav.position.y + distance * math.sin(ray_angle)
            pygame.draw.line(self.screen, (0, 255, 0), (self.uav.position.x * SCALE, VIEWPORT_H - self.uav.position.y * SCALE), (end_x * SCALE, VIEWPORT_H - end_y * SCALE))

        if self.uav:
            velocity_vector = self.uav.linearVelocity

            uav_center = (self.uav.position.x * SCALE, VIEWPORT_H - self.uav.position.y * SCALE)
            velocity_end = (uav_center[0] + velocity_vector.x, 
                            uav_center[1] - velocity_vector.y)  # Subtract y because of Pygame's y-axis direction

            pygame.draw.line(self.screen, (0, 0, 0), uav_center, velocity_end, 2) 

        pygame.display.flip()

    def close(self):
        if self.screen:
            pygame.quit()
            self.screen = None
            self.isopen = False

# Example usage
#env = SimpleUAVEnv()
#env.reset()
#env.render()



In [3]:
import time

# Initialize the environment
env = SimpleUAVEnv()
observation = env.reset()

# Parameters for the test
total_steps = 100  # Total number of steps in the test
thrust_increment = 1  # Increment in thrust per step
angular_momentum = 0  # Constant angular momentum (for simplicity)

# Run the test
for step in range(total_steps):
    # Gradually increase thrust
    thrust = thrust_increment * step

    # Create the action (thrust, angular momentum)
    action = [thrust, angular_momentum]

    # Perform a step in the environment
    state, reward, done, info = env.step(action)

    # Render the current state
    env.render()

    # Break the loop if the episode is done
    if done:
        break

    # Pause for a short time to see the animation
    time.sleep(1/FPS)

# Close the environment
env.close()


In [4]:
state

[-0.861605151494344,
 0.004823875427246094,
 3.719007968902588,
 2.6099803924560545,
 -0.11469008773565292,
 -3.3139469146728517,
 0.10043011198249736,
 0.09962835048900473,
 0.10176177258154455,
 0.10720912874244318,
 0.11707231464240389,
 0.1339429887680417,
 0.1648379557437965,
 0.24424602842904092,
 1.0,
 1.0]

## **NN in Jax/Flax**
pip install flax (CPU only)

In [5]:
OBS_ARR_START_IDX = 6 # The first six elements are the UAV's position, velocity, angle and angular velocity
STATE_SIZE = OBS_ARR_START_IDX + UAV_NUM_RAYS

class ANN(nn.Module):
    @nn.compact
    def __call__(self, x, input_dim, latent_dim, output_dim):
        x = nn.Dense(input_dim)(x)  
        x = nn.silu(x)        
        x = nn.Dense(latent_dim)(x)   
        x = nn.silu(x)
        x = nn.Dense(latent_dim)(x)
        x = nn.silu(x)
        x = nn.Dense(output_dim)(x)
        return x

class ANN(nn.Module):
    input_dim: int
    latent_dim: int
    output_dim: int

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(self.input_dim)(x)  
        x = nn.silu(x)        
        x = nn.Dense(self.latent_dim)(x)   
        x = nn.silu(x)
        x = nn.Dense(self.latent_dim)(x)
        x = nn.silu(x)
        x = nn.Dense(self.output_dim)(x)
        return x

key = jax.random.PRNGKey(0)
#model = ANN()
#params =model.init(key, jnp.ones((1, STATE_SIZE)), STATE_SIZE, 64, 2)
model = ANN(input_dim=STATE_SIZE, latent_dim=64, output_dim=2)
params =model.init(key, jnp.ones((1, STATE_SIZE)))

learning_rate = 1e-3
optimizer = optax.adam(learning_rate)

# Create the training state
state = train_state.TrainState.create(apply_fn=model.apply, params=params, tx=optimizer)

GAMMA = .99
DECAY_FACTOR = .99995
BATCH_SIZE = 64
#MAX_EPISODES = 800
MAX_EPISODES = 2
ACCEPTABLE_AVERAGE_SCORE_THRESHOLD = 190
MAX_ACCEPTABLE_AVG_SCORE_COUNTER = 100

def choose_action(model, state, epsilon, key):
    # Random action
    if np.random.rand() < epsilon:
        return np.random.randint(2), key

    # Predicted action
    q_values = model.apply({"params": state.params}, state)
    return np.argmax(q_values), key

# Memory for experience replay
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        return random.sample(self.buffer, min(len(self.buffer), batch_size))

@jax.jit
def train_step(state, batch):
    def loss_fn(params):
        states, actions, rewards, next_states = batch
        q_values = model.apply({"params": params}, states)
        next_q_values = model.apply({"params": params}, next_states)
        target_q_values = rewards + GAMMA * jnp.max(next_q_values, axis=1)
        chosen_q_values = jnp.choose(actions, q_values)
        loss = jnp.mean((chosen_q_values - target_q_values) ** 2)
        return loss

    grad_fn = jax.value_and_grad(loss_fn)
    loss, grads = grad_fn(state.params)
    return state.apply_gradients(grads=grads), loss


memory = ReplayBuffer(10000)  # Adjust size as needed


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
